## Alphabet Soup Charity - Model Optimization

---

### (1) Optimising the model by increasing the number of values for each bin & removing columns

Our original model includes 43 features which and inputs which may introduce high variance and low bias for our dataset, hance, manipulating the dataset by increasing the amount of values in the bins may be necessary and valueable to finetune our model. 

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df_1 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df_1.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-10-20 11:43:00.081144: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_1.drop(columns=['EIN', 'NAME', 'STATUS'], axis=1, inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df_1.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df_1['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace_1 = list(application_df_1['APPLICATION_TYPE'].value_counts().index)

# Replace in dataframe
for i, app in enumerate(application_types_to_replace_1):
    if i >= 8:
        application_df_1['APPLICATION_TYPE'] = application_df_1['APPLICATION_TYPE'].replace(app,"Rare Applications")
    elif i >5 and i <=7:
        application_df_1['APPLICATION_TYPE'] = application_df_1['APPLICATION_TYPE'].replace(app,"Less Common Applications")
    elif i >2 and i <=5:
        application_df_1['APPLICATION_TYPE'] = application_df_1['APPLICATION_TYPE'].replace(app,"Relativly Common Applications")
    else:
        continue

# # Check to make sure binning was successful
application_df_1['APPLICATION_TYPE'].value_counts()

T3                               27037
Relativly Common Applications     2975
T4                                1542
Less Common Applications          1253
T6                                1216
Rare Applications                  276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df_1['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Looking at CLASSIFICATION value counts >1
application_df_1['CLASSIFICATION'].value_counts().loc[application_df_1['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace_1 = list(application_df_1['CLASSIFICATION'].value_counts().index)

# Replace in dataframe
for i, cls in enumerate(classifications_to_replace_1):
    if i > 4:
        application_df_1['CLASSIFICATION'] = application_df_1['CLASSIFICATION'].replace(cls,"Other")
        pass
    elif i > 2 and i<=5:
        application_df_1['CLASSIFICATION'] = application_df_1['CLASSIFICATION'].replace(cls,"Less common government organisation")
    else:
        continue
    
# # Check to make sure binning was successful
application_df_1['CLASSIFICATION'].value_counts()

C1000                                  17326
C2000                                   6074
C1200                                   4837
Less common government organisation     3801
Other                                   2261
Name: CLASSIFICATION, dtype: int64

In [9]:
# Look at A value counts for binning
application_df_1['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of ORGANIZATION to be replaced
organisations_to_replace_1 = list(application_df_1['ORGANIZATION'].value_counts().index[2:])

# Replace in dataframe
for org in organisations_to_replace_1:
    application_df_1['ORGANIZATION'] = application_df_1['ORGANIZATION'].replace(org,"Other")
    
# Check to make sure binning was successful
application_df_1['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [11]:
# Look at AFFILIATION value counts for binning
application_df_1['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of AFFILIATION to be replaced
affiliation_to_replace_1 = list(application_df_1['AFFILIATION'].value_counts().index[2:])

# Replace in dataframe
for afl in affiliation_to_replace_1:
    application_df_1['AFFILIATION'] = application_df_1['AFFILIATION'].replace(afl,"Other")
    
# Check to make sure binning was successful
application_df_1['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df_1  = pd.get_dummies(application_df_1)

In [14]:
# Split our preprocessed data into our features and target arrays
X_1 = dummy_df_1.drop(columns='IS_SUCCESSFUL', axis=1)
y_1 = dummy_df_1['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, random_state=1)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler_1 = scaler.fit(X_train_1)

# Scale the data
X_train_scaled_1 = X_scaler_1.transform(X_train_1)
X_test_scaled_1 = X_scaler_1.transform(X_test_1)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_1_hidden_layer_1 = 80
model_1_hidden_layer_2 = 30
model_1_output_layer = 1 


nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=model_1_hidden_layer_1, activation='relu',input_dim=len(X_1.columns)))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=model_1_hidden_layer_2, activation='tanh'))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=model_1_output_layer, activation='sigmoid'))

# Check the structure of the model
nn_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2800      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


2023-10-20 11:43:04.562913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# Compile the model
nn_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define a callback to save the model's weights every five epochs
checkpoint_filepath_1 = 'model_weights.h5'
model_checkpoint_callback_1 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_1,
    save_weights_only=True,
    save_freq = 5 * len(X_train_scaled_1) // 32)  # Save every five epochs

In [18]:
# Train the model
nn_1.fit(X_test_scaled_1, y_test_1, epochs=100)

Epoch 1/100
268/268 [==============================] - 1s 779us/step - loss: 0.5949 - accuracy: 0.6988
Epoch 2/100
268/268 [==============================] - 0s 775us/step - loss: 0.5679 - accuracy: 0.7230
Epoch 3/100
268/268 [==============================] - 0s 750us/step - loss: 0.5632 - accuracy: 0.7257
Epoch 4/100
268/268 [==============================] - 0s 751us/step - loss: 0.5599 - accuracy: 0.7278
Epoch 5/100
268/268 [==============================] - 0s 733us/step - loss: 0.5588 - accuracy: 0.7266
Epoch 6/100
268/268 [==============================] - 0s 768us/step - loss: 0.5555 - accuracy: 0.7286
Epoch 7/100
268/268 [==============================] - 0s 802us/step - loss: 0.5541 - accuracy: 0.7301
Epoch 8/100
268/268 [==============================] - 0s 732us/step - loss: 0.5544 - accuracy: 0.7315
Epoch 9/100
268/268 [==============================] - 0s 772us/step - loss: 0.5507 - accuracy: 0.7340
Epoch 10/100
268/268 [==============================] - 0s 805us/step - l

In [19]:
# Evaluate the model using the test data
model_loss_1, model_accuracy_1 = nn_1.evaluate(X_test_scaled_1,y_test_1,verbose=2)
print(f"Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")

268/268 - 0s - loss: 0.5292 - accuracy: 0.7433 - 230ms/epoch - 859us/step
Loss: 0.5291615724563599, Accuracy: 0.7433236241340637


------

### (2) Optimising the model by increasing the apochs

In [20]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df_2 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df_2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_2.drop(columns=['EIN', 'NAME'], axis=1, inplace=True)

In [22]:
# Determine the number of unique values in each column.
application_df_2.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [23]:
# Look at APPLICATION_TYPE value counts for binning
application_df_2['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace_2 = list(application_df_2['APPLICATION_TYPE'].value_counts().index[8:])

# Replace in dataframe
for app in application_types_to_replace_2:
    application_df_2['APPLICATION_TYPE'] = application_df_2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [25]:
# Look at CLASSIFICATION value counts for binning
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [26]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df_2['CLASSIFICATION'].value_counts().loc[application_df_2['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [27]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace_2 = list(application_df_2['CLASSIFICATION'].value_counts().index[5:])

# Replace in dataframe
for cls in classifications_to_replace_2:
    application_df_2['CLASSIFICATION'] = application_df_2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [28]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df_2  = pd.get_dummies(application_df_2)

In [29]:
# Split our preprocessed data into our features and target arrays
X_2 = dummy_df_2.drop(columns='IS_SUCCESSFUL', axis=1)
y_2 = dummy_df_2['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, random_state=1)

In [31]:
# Create a StandardScaler instances
scaler_2 = StandardScaler()

# Fit the StandardScaler
X_scaler_2 = scaler_2.fit(X_train_2)

# Scale the data
X_train_scaled_2 = X_scaler_2.transform(X_train_2)
X_test_scaled_2 = X_scaler_2.transform(X_test_2)

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_2_hidden_layer_1 = 80
model_2_hidden_layer_2 = 30
model_2_output_layer = 1 


nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=model_2_hidden_layer_1, activation='relu',input_dim=len(X_2.columns)))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=model_2_hidden_layer_2, activation='tanh'))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=model_2_output_layer, activation='sigmoid'))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define a callback to save the model's weights every five epochs
checkpoint_filepath_2 = 'model_weights.h5'
model_checkpoint_callback_2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_2,
    save_weights_only=True,
    save_freq = 5 * len(X_train_scaled_2) // 32)  # Save every five epochs

In [34]:
# Train the model
nn_2.fit(X_test_scaled_2, y_test_2, epochs=200)

Epoch 1/200
268/268 [==============================] - 0s 813us/step - loss: 0.5855 - accuracy: 0.7124
Epoch 2/200
268/268 [==============================] - 0s 792us/step - loss: 0.5621 - accuracy: 0.7241
Epoch 3/200
268/268 [==============================] - 0s 776us/step - loss: 0.5580 - accuracy: 0.7269
Epoch 4/200
268/268 [==============================] - 0s 783us/step - loss: 0.5540 - accuracy: 0.7306
Epoch 5/200
268/268 [==============================] - 0s 762us/step - loss: 0.5498 - accuracy: 0.7313
Epoch 6/200
268/268 [==============================] - 0s 839us/step - loss: 0.5497 - accuracy: 0.7327
Epoch 7/200
268/268 [==============================] - 0s 761us/step - loss: 0.5478 - accuracy: 0.7318
Epoch 8/200
268/268 [==============================] - 0s 765us/step - loss: 0.5468 - accuracy: 0.7339
Epoch 9/200
268/268 [==============================] - 0s 758us/step - loss: 0.5457 - accuracy: 0.7348
Epoch 10/200
268/268 [==============================] - 0s 794us/step - l

In [36]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled_2,y_test_2,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 0s - loss: 0.5186 - accuracy: 0.7481 - 141ms/epoch - 525us/step
Loss: 0.51863694190979, Accuracy: 0.7481049299240112


As we can see our orignial model, without increasing the amount of values for the bins, performed better with the increased amount of apochs. The original model scored **0.7481** versus **0.7433** with the increased amount of values for each of the relevant bins.

----

### (3) Optimising the model by increasing the amount of hidden layers and neurons

In [37]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df_3 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df_3.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [38]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_3.drop(columns=['EIN', 'NAME'], axis=1, inplace=True)

In [39]:
# Determine the number of unique values in each column.
application_df_3.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [40]:
# Look at APPLICATION_TYPE value counts for binning
application_df_3['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [41]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace_3 = list(application_df_3['APPLICATION_TYPE'].value_counts().index[8:])

# Replace in dataframe
for app in application_types_to_replace_3:
    application_df_3['APPLICATION_TYPE'] = application_df_3['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_3['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [42]:
# Look at CLASSIFICATION value counts for binning
application_df_3['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [43]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df_3['CLASSIFICATION'].value_counts().loc[application_df_3['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [44]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace_3 = list(application_df_3['CLASSIFICATION'].value_counts().index[5:])

# Replace in dataframe
for cls in classifications_to_replace_3:
    application_df_3['CLASSIFICATION'] = application_df_3['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_3['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [45]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df_3 = pd.get_dummies(application_df_3)

In [46]:
# Split our preprocessed data into our features and target arrays
X_3 = dummy_df_3.drop(columns='IS_SUCCESSFUL', axis=1)
y_3 = dummy_df_3['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, random_state=1)

In [47]:
# Create a StandardScaler instances
scaler_3 = StandardScaler()

# Fit the StandardScaler
X_scaler_3 = scaler_3.fit(X_train_3)

# Scale the data
X_train_scaled_3 = X_scaler_3.transform(X_train_3)
X_test_scaled_3 = X_scaler_3.transform(X_test_3)

In [48]:
X_train_scaled_3.shape

(25724, 43)

In [49]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_3_hidden_layer_1 = 100
model_3_hidden_layer_2 = 40
model_3_hidden_layer_3 = 10
model_3_hidden_layer_4 = 3

model_3_output_layer = 1 


nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=model_3_hidden_layer_1, activation='relu',input_dim=len(X_3.columns)))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=model_3_hidden_layer_2, activation='tanh'))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=model_3_hidden_layer_3, activation='leaky_relu'))

# Forth hidden layer
nn_3.add(tf.keras.layers.Dense(units=model_3_hidden_layer_4, activation='leaky_relu'))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=model_3_output_layer, activation='sigmoid'))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               4400      
                                                                 
 dense_7 (Dense)             (None, 40)                4040      
                                                                 
 dense_8 (Dense)             (None, 10)                410       
                                                                 
 dense_9 (Dense)             (None, 3)                 33        
                                                                 
 dense_10 (Dense)            (None, 1)                 4         
                                                                 
Total params: 8,887
Trainable params: 8,887
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Compile the model
nn_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define a callback to save the model's weights every five epochs
checkpoint_filepath_3 = 'model_weights.h5'
model_checkpoint_callback_3 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_3,
    save_weights_only=True,
    save_freq = 5 * len(X_train_scaled_3) // 32)  # Save every five epochs

In [51]:
# Train the model
nn_3.fit(X_test_scaled_3, y_test_3, epochs=100)

Epoch 1/100
268/268 [==============================] - 1s 875us/step - loss: 0.6024 - accuracy: 0.7102
Epoch 2/100
268/268 [==============================] - 0s 873us/step - loss: 0.5646 - accuracy: 0.7286
Epoch 3/100
268/268 [==============================] - 0s 886us/step - loss: 0.5569 - accuracy: 0.7296
Epoch 4/100
268/268 [==============================] - 0s 1000us/step - loss: 0.5540 - accuracy: 0.7300
Epoch 5/100
268/268 [==============================] - 0s 894us/step - loss: 0.5510 - accuracy: 0.7332
Epoch 6/100
268/268 [==============================] - 0s 873us/step - loss: 0.5499 - accuracy: 0.7326
Epoch 7/100
268/268 [==============================] - 0s 872us/step - loss: 0.5480 - accuracy: 0.7342
Epoch 8/100
268/268 [==============================] - 0s 972us/step - loss: 0.5476 - accuracy: 0.7347
Epoch 9/100
268/268 [==============================] - 0s 897us/step - loss: 0.5457 - accuracy: 0.7336
Epoch 10/100
268/268 [==============================] - 0s 897us/step - 

In [52]:
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn_3.evaluate(X_test_scaled_3,y_test_3,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 0s - loss: 0.5208 - accuracy: 0.7471 - 229ms/epoch - 856us/step
Loss: 0.5208185911178589, Accuracy: 0.7470554113388062


Interestingly, even with an increased number of hidden layers, the accuracy score of the model didn't improve. The original model, with an increased number of epochs, scored slightly better with **0.7481** compared to **0.7470** for the model with the increased number of hidden layers. This could potentially be explained by the fact that increasing the number of hidden layers in our model also increases the complexity of the model.

----

### (4) Optimising the model by using different activation functions

In [53]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt


#  Import and read the charity_data.csv.
import pandas as pd 
application_df_4 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df_4.head()

Using TensorFlow backend


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [54]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_4.drop(columns=['EIN', 'NAME'], axis=1, inplace=True)

In [55]:
# Determine the number of unique values in each column.
application_df_4.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [56]:
# Look at APPLICATION_TYPE value counts for binning
application_df_4['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [57]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace_4 = list(application_df_4['APPLICATION_TYPE'].value_counts().index[8:])

# Replace in dataframe
for app in application_types_to_replace_4:
    application_df_4['APPLICATION_TYPE'] = application_df_4['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_4['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [58]:
# Look at CLASSIFICATION value counts for binning
application_df_4['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [59]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df_4['CLASSIFICATION'].value_counts().loc[application_df_4['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [60]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace_4 = list(application_df_4['CLASSIFICATION'].value_counts().index[5:])

# Replace in dataframe
for cls in classifications_to_replace_4:
    application_df_4['CLASSIFICATION'] = application_df_4['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_4['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [61]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df_4  = pd.get_dummies(application_df_4)

In [62]:
# Split our preprocessed data into our features and target arrays
X_4 = dummy_df_4.drop(columns='IS_SUCCESSFUL', axis=1)
y_4 = dummy_df_4['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, y_4, random_state=1)

In [63]:
# Create a StandardScaler instances
scaler_4 = StandardScaler()

# Fit the StandardScaler
X_scaler_4 = scaler_4.fit(X_train_4)

# Scale the data
X_train_scaled_4 = X_scaler_4.transform(X_train_4)
X_test_scaled_4 = X_scaler_4.transform(X_test_4)

In [64]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    # * instantiate a new Sequential model
    nn_model = tf.keras.Sequential()
    
    # * Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'leaky_relu'])

    # * Number of units for the first hidden layer (80 units in the provided model)
    nn_model.add(tf.keras.layers.Dense(units=80, activation=activation, input_dim=len(X_4.columns)))
    
    # * Number of units for the second hidden layer (30 units in the provided model)
    nn_model.add(tf.keras.layers.Dense(units=30, activation=activation))
    
    # * Adding the output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # * Complie the model
    nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

    return nn_model



In [65]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [66]:
# Run the kerastuner search for best hyperparameters with 100 epochs for each configuration
tuner.search(X_train_scaled_4,
             y_train_4, epochs=100,
             validation_data=(X_test_scaled_4, y_test_4),
             callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy', patience=10)])

In [67]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(4)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}
{'activation': 'relu', 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}
{'activation': 'leaky_relu', 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}
{'activation': 'sigmoid', 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}


In [69]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled_4,y_test_4,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7307 - 249ms/epoch - 930us/step
Loss: 0.5583305358886719, Accuracy: 0.7307288646697998
268/268 - 0s - loss: 0.5557 - accuracy: 0.7306 - 249ms/epoch - 931us/step
Loss: 0.5556564927101135, Accuracy: 0.7306122183799744
268/268 - 0s - loss: 0.5602 - accuracy: 0.7299 - 251ms/epoch - 936us/step
Loss: 0.5601853132247925, Accuracy: 0.729912519454956


In [70]:
# Get second best model hyperparameters
first_hyper = tuner.get_best_hyperparameters(2)[0]
first_hyper.values

{'activation': 'tanh',
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 4,
 'tuner/round': 0}

In [71]:
top_model = tuner.get_best_models(1)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled_4,y_test_4,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5583 - accuracy: 0.7307 - 246ms/epoch - 918us/step
Loss: 0.5583305358886719, Accuracy: 0.7307288646697998


As we can see, changing the activation functions in our model also didn't improve the performance of the model relative to the original model with increased amount of apochs. So we can conclude that the original model with the increased amount of apochs performed the best with accuracy score of **0.7470**.

In [72]:
# Export our optimised model to HDF5 file
nn_2.save('AlphabetSoupCharity_Optimisation.h5')